In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=32959c7546ae8cd3863ea98202c583c5e91e2aa3e3cda11402709836ff6e7018
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("/content/Summarizer_Data-Final.csv", error_bad_lines=False)

<ipython-input-5-397315c65263>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/Summarizer_Data-Final.csv", error_bad_lines=False)


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_summary, val_summary = train_test_split(
    df['content'], df['summary'], test_size=0.1)  # 20% for validation

In [ ]:
from transformers import PegasusTokenizer

model_name = 'google/pegasus-large'
tokenizer = PegasusTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

In [ ]:
train_texts = train_texts.fillna("").tolist()
train_summary = train_summary.fillna("").tolist()
val_texts = val_texts.fillna("").tolist()
val_summary = val_summary.fillna("").tolist()

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding='longest', max_length=1024, return_tensors="pt")
val_encodings = tokenizer(val_texts, truncation=True, padding='longest', max_length=1024, return_tensors="pt")
train_labels = tokenizer(train_summary, truncation=True, padding='longest', max_length=256, return_tensors="pt")
val_labels = tokenizer(val_summary, truncation=True, padding='longest', max_length=256, return_tensors="pt")

In [ ]:
import torch

class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SummaryDataset(train_encodings, train_labels)
val_dataset = SummaryDataset(val_encodings, val_labels)

In [ ]:
from transformers import PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
def generate_summaries(model, tokenizer, texts, device):
    model.to(device)
    summaries = []

    for text in texts:
        encoded_input = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        generated_ids = model.generate(encoded_input['input_ids'], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
        summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reference_summaries = [tokenizer.decode(labels, skip_special_tokens=True) for labels in val_labels['input_ids']]
generated_summaries = generate_summaries(model, tokenizer, val_texts, device)

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(ref, gen) for ref, gen in zip(reference_summaries, generated_summaries)]

In [ ]:
average_scores = {
    'rouge1': np.mean([score['rouge1'].fmeasure for score in rouge_scores]),
    'rouge2': np.mean([score['rouge2'].fmeasure for score in rouge_scores]),
    'rougeL': np.mean([score['rougeL'].fmeasure for score in rouge_scores])
}

print("Average ROUGE Scores:", average_scores)

Average ROUGE Scores: {'rouge1': 0.3928785433753455, 'rouge2': 0.23000755594174763, 'rougeL': 0.28300922053816174}


In [ ]:
texts_to_summarize = [

    """Climate change refers to significant, long-term changes in the global climate. The global climate is a connected system that is always in motion, and it is being affected by human activities. One of the most noticeable effects of climate change in the past century has been the increase in temperature around the world. The average global temperature has increased by about 1.1 to 1.2 degrees Celsius since 1900. This change has led to a wide range of impacts on the environment, ecosystems, and human societies. One of the primary causes of climate change is the release of greenhouse gases into the Earth's atmosphere. These gases, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), trap heat from the sun, leading to a warming effect known as the greenhouse effect. The majority of these emissions come from human activities, including the burning of fossil fuels for energy, deforestation, and industrial processes. The consequences of climate change are far-reaching and diverse. One of the most critical impacts is the rise in sea levels caused by the melting of polar ice caps and glaciers, as well as the expansion of seawater as it warms. This rise in sea levels poses a significant threat to coastal communities and islands. Additionally, climate change has been linked to more frequent and severe weather events, such as hurricanes, droughts, heatwaves, and heavy rainfall. Ecosystems are also being affected by climate change. Shifts in temperature and weather patterns can disrupt the natural habitats of many species, leading to changes in biodiversity. Some species may become extinct if they cannot adapt quickly enough to these changes. Furthermore, climate change can exacerbate existing environmental problems, such as habitat destruction and pollution, making it even harder for ecosystems to maintain their balance. The impacts of climate change extend to human societies as well. These impacts include threats to food and water supplies, increased risks to health, economic consequences, and potential displacement of populations. For instance, changes in precipitation patterns and temperature can affect crop yields, leading to food shortages and increased prices. Warmer temperatures can also contribute to the spread of diseases. Addressing climate change requires coordinated global action. This includes reducing greenhouse gas emissions, transitioning to renewable energy sources, and protecting and restoring forests. Additionally, societies need to adapt to the changes that are already underway. This involves building resilient infrastructure, developing sustainable agricultural practices, and planning for potential climate-related disasters. In conclusion, climate change is a complex and urgent issue that impacts the entire planet. It demands immediate and sustained action to mitigate its effects and safeguard the future of the environment and human societies."""
]

inputs = tokenizer(texts_to_summarize, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
# Move inputs to the same device as the model
inputs = inputs.to(model.device)

# Generate summaries
summary_ids = model.generate(inputs['input_ids'], max_length=256, num_beams=4, early_stopping=True)

# Decode generated summaries back to text
summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
for i, summary in enumerate(summaries):
    print(f"Summary {i+1}:\n{summary}\n")

Summary 1:
One of the most critical impacts is the rise in sea levels caused by the melting of polar ice caps and glaciers, as well as the expansion of seawater as it warms. Shifts in temperature and weather patterns can disrupt the natural habitats of many species, leading to changes in biodiversity. The impacts of climate change extend to human societies as well. For instance, changes in precipitation patterns and temperature can affect crop yields, leading to food shortages and increased prices.

